In [ ]:
import re

import janitor
import pandas as pd
from IPython.display import display
from PIL import Image, UnidentifiedImageError

In [ ]:
RAW_DATA = "./arte-urbana-manual.xlsx"
OUTPUT_DATA = "../src/data.json"
IMG_FOLDER = "../img/raw/pictures/"

In [ ]:
df = pd.read_excel(
    RAW_DATA,
    # index_col="ID",
    verbose=True,
    dtype={"Data": "Int64"},
)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
# df["Localizacao"].unique()
# df["Titulo"].value_counts(dropna=False)
# df["Freguesia"].value_counts(dropna=False)
# df["Freguesia"].unique()
df["Tipologia"].unique()

http://geoportal.cm-amadora.pt/graffiti/Arte_Urbana/index.html

**Quotation marks**

- «
- »
- “
- ”

In [ ]:
df["Descricao_manual"].to_list()

In [ ]:
df.isna().sum()

In [ ]:
df = df.clean_names()

In [ ]:
df.columns

In [ ]:
# df[df["autoria"].str.startswith("APAURB")]
df[df["autoria"].str.contains("Eiffel")]

In [ ]:
df.loc[df["autoria"].str.startswith("APAURB"), "autoria"] = "APAURB"

In [ ]:
df.loc[df["autoria"].str.contains("gvs", flags=re.IGNORECASE), "autoria"] = df.loc[
    df["autoria"].str.contains("gvs", flags=re.IGNORECASE), "autoria"
].str.replace("gvs", "GVS", flags=re.IGNORECASE, regex=False)

In [ ]:
df.loc[df["autoria"] == "VFM/GVS", "autoria"] = "VFM/GVS Crews"

In [ ]:
df.loc[df["autoria"].str.contains("Gustavo Eiffel"), "autoria"] = df.loc[
    df["autoria"].str.contains("Gustavo Eiffel"), "autoria"
].str.replace("Gustavo Eiffel", "Gustave Eiffel", regex=False)

In [ ]:
# Aka Corleone
# https://regex101.com/

df.loc[
    df["autoria"].str.match("aka\s?corleone", flags=re.IGNORECASE), "autoria"
] = "Aka Corleone"

In [ ]:
# df.loc[df["autoria"].str.match("aka\s?corleone", flags=re.IGNORECASE)]

In [ ]:
title = "Plasticus Maritimus"

df.loc[(df["autoria"] == "Tiago Albuquerque") & (df["id"] == 137), "titulo"] = title

In [ ]:
df["titulo"] = df["titulo"].str.strip()
df["autoria"] = df["autoria"].str.strip()
df["localizacao"] = df["localizacao"].str.strip()
# df["descricao_manual"] = df["descricao_manual"].str.strip()

In [ ]:
df.tail()

In [ ]:
df["autoria"].nunique()

In [ ]:
with pd.option_context("display.max_rows", None):
    display(df["autoria"].value_counts(dropna=False))

In [ ]:
df = df.drop("descricao", axis=1)

In [ ]:
# https://pillow.readthedocs.io/en/stable/reference/Image.html#PIL.Image.Image.size
sizes = []
missing_size = (0, 0)

for pic_id in df["id"]:
    try:
        with Image.open(f"{IMG_FOLDER}id{pic_id}.jpg") as im:
            sizes.append(im.size)  # (width, height)
    except UnidentifiedImageError:
        sizes.append(missing_size)
        print(f"✘ id{pic_id}.jpg")

In [ ]:
df[["pic_width", "pic_height"]] = sizes

In [ ]:
df.head()
# df.query("id == 89")

In [ ]:
df = df.sort_values("id", ignore_index=True)

In [ ]:
indent = 4

df.to_json(OUTPUT_DATA, orient="records", force_ascii=False, indent=indent)

---